In [1]:

from mindquantum.algorithm.nisq import HardwareEfficientAnsatz,RYFull
from mindquantum.core.parameterresolver import  PRGenerator
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, I,CNOT
from mindquantum.core.circuit import Circuit,UN
import mindspore as ms
from mindquantum.core.parameterresolver import PRGenerator
import random
from mindspore import Tensor,ops
import tensorcircuit as tc
import tensorflow as tf
import mindspore.numpy as mnp
from DQAS_tool import wash_pr,Mindspore_ansatz_micro,vag_nnp_micro
import sys
from typing import Union
sys.path.append('..')
from Test_tool import Test_ansatz
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.core.circuit import change_param_name,apply

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [2]:
pr_pool = PRGenerator('pool')
parameterized_circuit= \
[UN(RX(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RX(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RX(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RX(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RY(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RY(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RY(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RY(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RZ(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RZ(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RZ(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RZ(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),]


unparameterized_circuit = \
[UN(X,maps_obj=[0],maps_ctrl=[1]),
 UN(X,maps_obj=[1],maps_ctrl=[0]),
 UN(Z,maps_obj=[0],maps_ctrl=[1]),
 UN(Z,maps_obj=[1],maps_ctrl=[0]),
 UN(Y,maps_obj=[0],maps_ctrl=[1]),
 UN(Y,maps_obj=[1],maps_ctrl=[0]),
 ]
ansatz_pr = PRGenerator('ansatz')
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=5
shape_nnp = (7,num_layer,shape_parametized)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [9]:
def Mindspore_ansatz_micro(Structure_p:np.array,
                     parameterized_pool:list[Circuit],
                     unparameterized_pool:list[Circuit],
                     num_layer:int=6,
                     n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    更新了非参数化门的算符池引入
    Structure_p:np.array DQAS中的权重参数,
    Ansatz_p:np.array  DQAS中的Ansatz参数,
    """
    if Structure_p.shape[0] != num_layer:
        raise ValueError('Structure_p shape must be equal to num_layer')
    
    if Structure_p.shape[1] != len(parameterized_pool)+len(unparameterized_pool):
        raise ValueError('Structure_p shape must be equal to size of pool')

    if isinstance(Structure_p, np.ndarray):
        my_stp = ms.Tensor(Structure_p, ms.float32)
    else:
        my_stp = Structure_p
        
    prg = PRGenerator('encoder')
    nqbits = n_qbits
    encoder = Circuit()
    # encoder += UN(H, nqbits)                                 
    for i in range(nqbits):                                  
        encoder += RY(prg.new()).on(i)    
    encoder = encoder.as_encoder()             
        
    sub_ansatz = Circuit()
    #print(my_stp.shape)
    for layer_index in range(my_stp.shape[0]):
        for op_index in range(my_stp.shape[1]):
            if my_stp[layer_index,op_index] == 0:
                continue
            if op_index < len(parameterized_pool):
                before_ansatz = parameterized_pool[op_index]
                before_ansatz = change_param_name(circuit_fn=before_ansatz,name_map={before_ansatz.ansatz_params_name[0]:f'ansatz{layer_index}'})
                sub_ansatz += before_ansatz
            else:
                sub_ansatz += unparameterized_pool[op_index-len(parameterized_pool)]
    
    whole_ansatz = Circuit()
    whole_ansatz += wash_pr(apply(sub_ansatz,[0,1]),index=0)
    whole_ansatz += wash_pr(apply(sub_ansatz,[2,3]),index=1)
    whole_ansatz += wash_pr(apply(sub_ansatz,[4,5]),index=2)
    whole_ansatz += wash_pr(apply(sub_ansatz,[6,7]),index=3)
    whole_ansatz += wash_pr(apply(sub_ansatz,[0,2]),index=4)
    whole_ansatz += wash_pr(apply(sub_ansatz,[4,6]),index=5)
    whole_ansatz += wash_pr(apply(sub_ansatz,[0,4]),index=6)
    whole_ansatz =  wash_pr(whole_ansatz,index=None)
                
    finnal_ansatz = encoder.as_encoder() + whole_ansatz.as_ansatz()
    return finnal_ansatz

In [3]:
def best_from_structure(structures: np.array)->Tensor:
    prob = ops.Softmax()(ms.Tensor(structures))
    return ops.Argmax(axis=-1)(prob)

def best_from_structure(structures: np.array)->Tensor:
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [4]:
ops_onehot = ops.OneHot(axis=-1)
stp_onehot = best_from_structure(stp)
stp_onehot = ops_onehot(ms.Tensor(stp_onehot),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
ansatz = Mindspore_ansatz_micro(Structure_p=stp_onehot,parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,num_layer=num_layer,n_qbits=8)

In [ ]:
best_from_structure(stp)

In [6]:
nnp.shape

(7, 5, 12)

In [67]:
def zeroslike_grad_nnp_micro(batch_sturcture: Union[np.ndarray, ms.Tensor], grad_nnp: Union[np.ndarray, ms.Tensor], shape_parametized: int, ansatz_parameters: np.ndarray) -> np.ndarray:
    '''
    用于根据算出的梯度更新ansatz参数    
    '''
    if isinstance(batch_sturcture, np.ndarray):
        mystp = ms.Tensor(batch_sturcture, ms.float32)
    else:
        mystp = batch_sturcture  # 如果 batch_sturcture 已经是 ms.Tensor 类型

    op_index = [ops.Argmax()(i) for i in mystp]
    print(op_index)
    zeros_grad_nnp = np.zeros_like(ansatz_parameters)
    count = 0
    for each_sub in range(7):
        for index,i in enumerate(op_index):
            if i >= shape_parametized:
                continue
            zeros_grad_nnp[each_sub, i] = grad_nnp[count]
            count += 1
        
    return zeros_grad_nnp

In [5]:
from DQAS_tool import  sampling_from_structure,zeroslike_grad_nnp_micro,nmf_gradient,vag_nnp_micro,DQAS_accuracy
#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.1))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 50
batch_size=256
avcost1 = 0
ops_onehot = ops.OneHot(axis=-1)
batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率

 
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp_micro(Structure_params=i,
                                    Ansatz_params=nnp,
                                    paramerterized_pool=parameterized_circuit,  unparamerterized_pool=unparameterized_circuit,
                                    num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp_micro(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)

      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    # print(f'loss={infd}\ngrad_nnp={gnnp}\ngrandient_stp={gstp_averge}')
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    cand_preset = best_from_structure(stp)
    best_candidates_history.append(cand_preset.asnumpy())
    

    if epoch % 1 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
        test_ansatz = Mindspore_ansatz_micro(Structure_p=stp_for_test,
                                            parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,
                                            num_layer=num_layer,
                                            n_qbits=8)
        
        ansatz_parameters=[]
        for layerIndex,i in enumerate(cand_preset):
            if i >=len(parameterized_circuit):
                continue
            else:
                ansatz_parameters.append(nnp[layerIndex,i])
        
        acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
        acc_history.append(acc)
        print(f'二分类准确率 Acc ={acc*100}% ')
        
        

0 0 1
0 1 0
0 2 3
0 3 2
1 0 1
1 1 0
1 2 3
1 3 2
2 0 1
2 1 0
2 2 3
2 3 2
3 0 1
3 1 0
3 2 3
3 3 2
4 0 1
4 1 0
4 2 3
4 3 2
5 0 1
5 1 0
5 2 3
5 3 2
6 0 1
6 1 0
6 2 3
6 3 2
[Tensor(shape=[], dtype=Int32, value= 1), Tensor(shape=[], dtype=Int32, value= 0), Tensor(shape=[], dtype=Int32, value= 3), Tensor(shape=[], dtype=Int32, value= 2), Tensor(shape=[], dtype=Int32, value= 16)]
0 0 8
0 1 8
0 2 4
1 0 8
1 1 8
1 2 4
2 0 8
2 1 8
2 2 4
3 0 8
3 1 8
3 2 4
4 0 8
4 1 8
4 2 4
5 0 8
5 1 8
5 2 4
6 0 8
6 1 8
6 2 4
[Tensor(shape=[], dtype=Int32, value= 8), Tensor(shape=[], dtype=Int32, value= 8), Tensor(shape=[], dtype=Int32, value= 4), Tensor(shape=[], dtype=Int32, value= 12), Tensor(shape=[], dtype=Int32, value= 12)]


IndexError: index 8 is out of bounds for axis 1 with size 5